In [1]:
import requests #pip install requests
from bs4 import BeautifulSoup #pip install bs4
import os
import time
import json
import re
import pandas as pd

API's used:
- Beautiful Soup: https://www.crummy.com/software/BeautifulSoup/bs4/doc/
- Requests : https://requests.readthedocs.io/en/master/

Futher Work:
- Database: https://medium.com/analytics-vidhya/tutorial-amazon-price-tracker-using-python-and-mongodb-part-1-aece6347ec63
- Proxy Servers, Exceptions, crawlers : https://blog.hartleybrody.com/scrape-amazon/

In [2]:
url= "https://www.ebay.com/itm/Reebok-Jet-Dashride-Running-Shoes-Sz-10-Mens-Silver-Gray-1AP502316-Athletic/143437209311?epid=14036828504&_trkparms=ispr%3D1&hash=item2165861edf:g:XlcAAOSwJO9dPZnn&enc=AQAFAAACYBaobrjLl8XobRIiIML1V4Imu%2Fn%2BzU5L90Z278x5ickk7d4nremBkvNKtcC0ZwqXDK%2Fkcm30YG4tTXkJ14lvXIPYHVuhiF1rDrBfbQSyjSYIyfMw5MnR3%2Bh7QffsplfYJlPW9yCK%2BKWyTAmYhku5vGW6JNt91CCdIrc0Twa6Ai7%2BwinIg7YdD3I6eP9xh8aFKVH1lqojT5k5AKSfCt9Uu%2FXHfyZvkYhIsZIhMxNe0gzKasMDuSvd14lHJNsiLPV1EmSyfT1%2B%2BRU0lv9AuiWvmGby3JMQzTBoBSgYNMVp6IfTqTMzdm58W%2BY8KcLuWO9JdeslB7ejS7r00qGXrRJMEVpgYGmK5YmG3RrfR3DuHSKg5F3%2Bo%2FlF9v5wTIvl6HgdfzljiVN3mXJFTGpOhAlzChAzE1d%2F7KOKwL4u0ArRx73CutPhydyWuZZCATgoFmZPuHxzqWysolyF8DjEFYe6QFuv2CiqqyqPzhNcT%2BzBSCfGoEvgB7FrJ62bfarlPgTENv3zcJJZCUuo8nzoJcvqCXn5eTLEM5VBey6fTzzwmuCPSZivuomh%2BFrXAWme%2FA2Ey43CTtc7jECiX2Oogb53DwuV3decq4g46YmAp%2BctyVMsK7PISg6XtNOkdL7LgzK9SimbMKPOYtmV7qpCZLqHKGpWT6W9HcnPcRAbEwn5y8BO5mViJkP6%2F2S30rdKFYyOVQxLYo4nyJGJbohjwT0idzS%2FuzUyZ4gQK%2BfyceTePUrBkeOtwf2aEAydkof4s7LNmHdzygfNM0MF%2FXyM35OCPKTlgcjGqhMPSFX4oyumJPf2&checksum=1434372093119a099adbb1b247eaaedbbd8302b7be5d"

In [3]:
headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36 Edg/84.0.522.40"}


In [4]:
def ParseHTML(url, printHTML = False):
    r = requests.get(url,headers=headers)
        
    if r.status_code != 200:        
        print("Page %s must have been blocked by eBay as the status code was %d"%(url,r.status_code))
        return None
    
    print("Successfully parsed page")
    soup = BeautifulSoup(r.content, features="lxml")
    
    if (printHTML):
        print(soup)
    
    return soup

In [5]:
html = ParseHTML(url, False)

Successfully parsed page


In [6]:
def GetProductName_eBay(html):
    nameTag = str(html.find(id='itemTitle'))
    name = re.search("</span>(.*)</h1>", nameTag).group(1)    
    return name

In [7]:
GetProductName_eBay(html)

'Reebok Jet Dashride Running Shoes Sz 10 Mens Silver Gray 1AP502316 Athletic'

In [21]:
def GetProductPrice_eBay(html):
    priceTag = str(html.find(id='prcIsum'))
    #print(priceTag)
    price = float(re.search("\$(.*)</span>", priceTag).group(1))
    print(price)
    return price

In [22]:
GetProductPrice_eBay(html)

95.75


95.75

In [19]:
def GetSimiliarProducts_ebay(html):
    try:
        suggestionsHtml = html.find("ul", attrs={"class": ['mfe-recos', 'mfe-recos-slider', 'clear', 'limit-small-screen']}).find_all("li")
        suggestionsDict = {}

        for suggestion in suggestionsHtml:
            url = suggestion.find("a")['href']
            name = suggestion.find("p").text
            suggestionsDict[name] = url

        return suggestionsDict
    except:
        return 'Sorry, no similar products found.'

In [20]:
GetSimiliarProducts_ebay(html)

'Sorry, no similar products found.'

In [17]:
def MonitorPrice(url, maximumPrice = 40, maximumTime = 10):
    
    # Parses the html to obtain the product name and current price
    html = ParseHTML(url)
    productPrice = GetProductPrice_eBay(html)
    productName = GetProductName_eBay(html)
    
    # sanity check
    if (productPrice < maximumPrice):
        return " GO BUY"

    print("Monioting product: " + productName + " \n")
    startTime = time.time()
    
    
    # Two conditions to keep checking:
        # 1. The product price is higher than what we are willing to pay
        # 2. The time we have spend running the bot is less than the maximumTime
    while ( productPrice > maximumPrice and time.time()-startTime < maximumTime):

        html = ParseHTML(url)
        productPrice = GetProductPrice_eBay(html)
        
        print(" Time: " + str(time.ctime()) )
        print(" Price: " + str(productPrice) + " \n")
        
        if (productPrice < maximumPrice):
            return "GO BUY!!!"
        
        # Wait for 3 seconds before checking again
        time.sleep(3)

        
    similiarProducts = GetSimiliarProducts_ebay(html)
    print("Product price did not drop to required value in given time frame. \n")
    print("Some similiar products:")
    
    if type(similarProducts) != str:
        for item in similiarProducts.keys():
            print(item)
            print(similiarProducts[item])
            print()
    
    return "Sorry"

In [23]:
MonitorPrice(url, 30, 10)

Successfully parsed page
95.75
Monioting product: Reebok Jet Dashride Running Shoes Sz 10 Mens Silver Gray 1AP502316 Athletic 

Successfully parsed page
95.75
 Time: Sat Oct  3 14:49:52 2020
 Price: 95.75 

Successfully parsed page
95.75
 Time: Sat Oct  3 14:49:58 2020
 Price: 95.75 

Product price did not drop to required value in given time frame. 

Some similiar products:


'Sorry'